In [ ]:
import numpy as np
import pandas
import platypus.io.logs
import os

In [ ]:
# Import the data from the specified logfile
log_filename = '../logs/platypus_20161126_'
data = platypus.io.logs.load(log_filename)

In [ ]:
# Extract position of boat and remove duplicate entries.
data["pose"]["time"] = data["pose"].index
position = data['pose'][['time', 'latitude', 'longitude']].drop_duplicates(subset='time', keep='first')
position = position[['latitude', 'longitude']]  # remove unneeded time column after de-duplication

# Put together all the sensor data we are looking for.
# This also combines the name of the sensor with the name of each channel.
sensor_names = ['ATLAS_DO', 'ES2', 'HDS_TEMP', 'HDS_DEPTH']
sensor_frames = []
for sensor_name in sensor_names:
    sensor_data = data[sensor_name].copy()
    sensor_data.columns = [ sensor_name + '_' + str(k) for k in sensor_data.columns ]
    sensor_frames.append(sensor_data)
sensor_data = pandas.concat(sensor_frames, axis=1)

# Find the position for each sensor reading.
sensor_position = position.reindex(sensor_data.index, method='nearest')
output = pandas.concat((sensor_position, sensor_data), axis=1)

# Fill in missing values with last known values.
output = output.apply(pandas.Series.interpolate, method='nearest')
output['epoch_time'] = output.index.astype(np.int64)

In [ ]:
# Output the file as a CSV in the same directory as the original logfile.
output_filename = os.path.splitext(log_filename)[0] + '.csv'
output.to_csv(output_filename, index=True)